In [1]:
import sys; sys.path.append('../..')
import random
import pyzx as zx
import os
import pickle
import time

In [2]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

In [3]:
def sliding_window(elements, window_size):
    if len(elements) < window_size:
        return [elements]
    
    for i in range(len(elements) - window_size + 1):
        yield elements[i:i+window_size]

### Heuristic simplification
- When simplifying ZX-diagrams with T-spiders, simplification routines like full_reduce lead to a very high two-qubit gate count.
- When using heuristic-based approaches we can circumvent the problem to some extent leading to better overall circuit cost after optimization

In [4]:
random.seed(1342)
g = zx.generate.cliffordT(qubits=10, depth=500, p_t=0.3, p_cnot=0.5)
c = zx.Circuit.from_graph(g)
c = zx.optimize.basic_optimization(c.split_phase_gates()).split_phase_gates()
print(c.stats())

Circuit  on 10 qubits with 468 gates.
        56 is the T-count
        412 Cliffords among which
        252 2-qubit gates (239 CNOT, 13 other) and
        50 Hadamard gates.


In [5]:
g = c.to_graph()
g_tele = zx.simplify.teleport_reduce(g)
g_tele.track_phases = False
print(zx.Circuit.from_graph(g).split_phase_gates().stats())

Circuit  on 10 qubits with 468 gates.
        56 is the T-count
        412 Cliffords among which
        252 2-qubit gates (239 CNOT, 13 other) and
        50 Hadamard gates.


In [6]:
g_full = g_tele.copy()
zx.simplify.full_reduce(g_full)
print(zx.extract_circuit(g_full.copy()).stats())

Circuit  on 10 qubits with 449 gates.
        56 is the T-count
        393 Cliffords among which
        293 2-qubit gates (82 CNOT, 211 other) and
        94 Hadamard gates.


In [15]:
applied_matches_la = []

for la in range(0,4):
    print("Lookahead:", la)
    start_time = time.time()
    g_greedy_la = g_tele.copy()
    iterations, applied_matches = zx.simplify.greedy_simp(g_greedy_la, lookahead=la)
    applied_matches_la.append(applied_matches)
    print(zx.extract_circuit(g_greedy_la.copy()).stats())
    print("Time:", time.time()-start_time)
    print("---------------------------------")

Lookahead: 0
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\wsajk\Documents\Arbeit\MUNIQC-Atoms\muniqc\Lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\wsajk\AppData\Local\Temp\ipykernel_42580\92727332.py", line 7, in <module>
    iterations, applied_matches = zx.simplify.greedy_simp(g_greedy_la, lookahead=la)
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\wsajk\Documents\Arbeit\MUNIQC-Atoms\pyzx-heuristics\demos\heuristic_demos\../..\pyzx\simplify.py", line 236, in greedy_simp
    greedy_wire_reduce_count, applied_matches = greedy_wire_reduce(g, filter_flow_func=filter_flow_func, include_boundaries=include_boundaries, include_gadgets=include_gadgets, max_vertex_index=max_vertex_index, threshold=threshold, lookahead=lookahead, quiet=quiet, stats=stats)
                                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
first_item = applied_matches_la[0]

for index, applied_matches in enumerate(applied_matches_la[1:], start=1):
    differences = [i for i in range(min(len(first_item), len(applied_matches))) if first_item[i] != applied_matches[i]]
    # differences = [i for i, element in enumerate(first_item) if element not in applied_matches] + [i for i, element in enumerate(applied_matches) if element not in first_item]
    if differences:
        first_diff_index = differences[0]
        print("Items in list 0 beginning at " + str(first_diff_index) + " :", first_item[first_diff_index:first_diff_index+index+1])
        print("Items in list " + str(index) + " beginning at " + str(first_diff_index) + " :", applied_matches[first_diff_index:first_diff_index+index+1])
        print("")

In [7]:
print("Causal Flow")
start_time = time.time()
g_greedy_cflow = g_tele.copy()
iterations, applied_matches = zx.simplify.greedy_simp(g_greedy_cflow, lookahead=1, filter_flow_func=zx.heuristics.flow_calculation.identify_causal_flow)
print(zx.extract_circuit(g_greedy_cflow.copy()).stats())
print("Time:", time.time()-start_time)
print("---------------------------------")

Causal Flow
Circuit  on 10 qubits with 693 gates.
        56 is the T-count
        637 Cliffords among which
        227 2-qubit gates (0 CNOT, 227 other) and
        322 Hadamard gates.
Time: 2.020843505859375
---------------------------------


In [ ]:
flow = zx.heuristics.flow_calculation.identify_causal_flow(g_greedy_cflow)
print(flow)

In [ ]:
g_nu = g_tele.copy()
zx.simplify.greedy_simp_neighbors(g_nu)
print(zx.extract_circuit(g_nu.copy()).stats())

In [ ]:
g_sim_n = g_tele.copy()
zx.simplify.sim_anneal_simp_neighbors(g_sim_n)
print(zx.extract_circuit(g_sim_n.copy()).stats())

In [ ]:
g_sim = g_tele.copy()
zx.simplify.sim_anneal_simp(g_sim)
print(zx.extract_circuit(g_sim.copy()).stats())